# 分散ブロックチェーンの実装

- 目標は分散ブロックチェーンで各ノードが、ドローンの離着陸地点の予約を管理してるポイントだとして、ドローンの着陸離陸許可がされるさまをつくる事

- [Bitcoin論文](https://bitcoin.org/bitcoin.pdf)をもとにblockchainの実装をする
    - 天下り的にならないよう，徐々にシステムをパワーアップさせていく方針で作る
    - 目標はシンプルなコンセンサスアルゴリズムの実装まで行い，ノード同士が競ってマイニングする様子を眺めること
    - 完成形はこんな感じ
    
    <img src="./demo.gif" width=50%>

# 信用のある第三者を介する取引
- まずは第三者を介した取引をシミュレーションしてみる
- 第三者は取引の公正さのすべてを請け負う

## 取引はどのように行うか? - Transactionの実装
- 取引記録を発行することで，取引を行う
    - Python3.7から実装された[dataclasses](https://docs.python.org/ja/3/library/dataclasses.html#)を使ってみる

In [1]:
from __future__ import annotations
from dataclasses import dataclass

In [2]:
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
import binascii

import json
import dataclasses as dc
from typing import List

import multiprocessing as mp
import random
from itertools import count
from time import sleep

from typing import Tuple, Sequence
import hashlib

from time import time

In [3]:
import sys
sys.path.append('src/')

In [4]:
from block import Block
from blockchain import BlockChain
from network import Network
from wallet import Wallet
from smartcontract import Smartcontract

- 個人の識別を`address`で行う
- 取引額を`value`とする

- senderはTransactionを発行して取引を行う

- Aliceがtransactionを発行してBankに登録することで，取引が成立する

- the_leader_nodeがトランザクションを発行して…等、解釈の変更が必要 

- ここで例えば，第三者の信頼度がちょっと落ちたとする
    - 取引内容の改ざんはどのように防げばよいだろうか?

# 電子署名を用いたtransactionの発行
- senderはtransactionに電子署名を行う
    - 今回はRSAを用いることにする
        - BitCoinは[ECDSA](https://en.bitcoin.it/wiki/Transaction#Input)を用いている
    - これによってtransactionの改ざんを検知することができる
- 電子署名の公開鍵はどのように扱う?
    - 公開鍵をWalletのaddressとする!

## Transactionの改良 - 電子署名フィールドの追加
- 電子署名できるようにする
    - `sign`を追加
    - 署名の際，transactionをstrで表すための`str_data` methodも追加
- ついでにデータをjson stringに変換するmethodを追加する
    - これは後々データのやり取りを簡単にするため

## Walletの改良 - 電子署名機能の追加
- RSAによる電子署名機能を実装する
    - pycryptoを用いる
        - 使い方はこれを参照 ([python \- Signing and verifying data using pycrypto \(RSA\) \- Stack Overflow](https://stackoverflow.com/questions/4232389/signing-and-verifying-data-using-pycrypto-rsa))
- 引き続きWalletはaddressを1つのみ持つことにする
    - 例えばBitCoinでは匿名性の向上のため，一つのWalletは複数のキーペアを持つことができる ([参考](https://en.bitcoin.it/wiki/Transaction))

- pycryptoのkey instanceをstrに変換するhelper functionを実装する
    - strにしておく理由は後ほどわかる

Walletの改良

- 秘密鍵`private_key`の追加 
- addressは対応する公開鍵とする
- `sign_transaction`の追加
    - transactionを受け取り，電子署名付きtransactionを返す
- `send`を変更
    - 電子署名付きtransactionを生成できるようになった!

つぎに，transactionの電子署名をチェックするhelper functionを作る
- signatureのpublic keyはtransactionのaddressなので，transactionを見れば電子署名の正当性を判別できる

## デモ: RSAを用いたtransactionの発行とチェック
先ほどと同じようにデモをしてみる

- transactionを発行し，電子署名を行い，bankに提出する

- transactionの電子署名を確認してみる

- データをちょと変えてみると，不正なtransactionになるのがわかる

- これは後にできると面白いかもしれない



- transactionの改竄リスクを低減することができた 
    - しかし依然として，**double-spending problem** を防ぐには信頼のある第三者が必要となる
    - これはデータで通貨を作る際の最も大きな障害だった(データは簡単にコピーできるから)
- すべてのtransactionが公開されていれば，double-spendingは防げるのでは？
    - 公開されていればreceiverも記録をチェックできる

# Timestamp Server によるTransactionの公開
- いくつかのtransactionをまとめて，timestampをつけ，**block**で公開することを考える
    - 信頼のある**timestamp server**がtransactionをまとめてtimestampをつけ，電子署名をして公開する
    - ある特定の時刻にそのtransactionが存在したことが保証される
- さらに，blockには直前のtimestampのhashを含め，**blockchain**を作る
    - blockを改ざんすると，以降のblockは全て不正なものとなる
- これでtransactionをチェックしてdouble-spendingを防ぐ第三者は必要なくなった
    - transactionは公開されているから，誰でも検証することができる

## Blockの実装
- strで表現できるよう，json methodを追加する
- ついでにhash methodも実装する

In [5]:
# from typing import Tuple, Sequence
# import hashlib

## Timestamp Serverの実装
- timestamp serverは以下の流れで**blockchain**を作る
    1. transactionのリストを受け取る
    2. 一つ前のblockのhashとtimestampをつけてBlockにまとめる
    3. **電子署名**をする
    4. 公開する

- blockの正当性の保証はtimestamp serverの**電子署名**で行う
    - serverに RSA キーペアをもたせる
- 最初のblockはprevious blockがないため，特別なブロック(**genesis**)を用意する ([参考](https://en.bitcoin.it/wiki/Genesis_block))
- blockchainは今回はただのリストにする

In [6]:
# from time import time

## blockchainの正当性確認
- blockchainの正当性を確認するhelper functionを用意する
    1. blockの持つ**previous_hash**は正しいか?
    2. blockに入っているtransactionは正当か?
    3. timestamp serverがつけた電子署名は正当か?

- blockの正当性確認

- blockchainの正当性確認

# Proof-of-Workの実装
- これがblockchainの革新的要素
- 先程は"信頼の置ける" timestamp serverの電子署名が，Blockの正当性を保証していた
- では誰でも同じ署名ができれば，誰でもtimestamp serverの仕事ができるのでは?
    - 電子署名でやると，当然ブロックの正当性の証明にはならない..
- **[HashCash](https://en.bitcoin.it/wiki/Hashcash)**を用いる

## Blockにnonceを加える
- これがtimestamp serverの電子署名の代わりとなる
- blockのハッシュ値が適当な条件を満たすように**nonce**を追加する
    - 例えば上4桁が0
    - この制約を**difficulty**と呼ぶ
    - difficultyは適当に調整される. 例えばbitcoinの場合は，blockの追加のintervalが10分程度になるように調整されるようだ
        - [Difficulty \- Bitcoin Wiki](https://en.bitcoin.it/wiki/Difficulty)
    - 今回はdifficultyは固定にする
- nonceを見つける作業が**mine**
    - nonceを見つけるのは難しい(たくさん試して，あたりを引くしかない)
    - これによってblockの生成を非常に高コストにすることができる
    - nonceが正しいことを確認するのは簡単
        - 電子署名と同様, 正当性の確認が簡単に行える
- **mine**は誰でも行える

- Blockにnonceをつける．今回の変更はこれだけ

## nonce の正当性確認functionを実装

## mineの実装
- あたりを引くまで手当たり次第試す

- ついでにverify_blockも改良

- BlockChainもちょっと改良
    - 自身をstrに変換するmethodをつける

## デモ: tranasctionの発行からminingまで

# ＊＊＊＊＊NodeとConsensus Algorithm
- 前回までで分散管理の準備は整った
- 具体的にblockchainをつくる**node**を実装する
- nodeやsender, receiverが集まって**network**を形成する
- networkの動きは以下のようになる

1. senderがtransaction生成．なるべく多くのnodeにbroadcastする（全てのnodeにbroadcastする事を確認する）
2. nodeはtransactionを集めてblockにまとめる
3. mine（必要ない）
4. 他のnodeにマイニングしたblockをbroadcastする
5. blockを受け取ったnodeは，blockの正当性を確認し，double-spendingのチェックをする（チェックはいくらしても良い、必要に応じて外して行く）
6. チェックが通れば，次のblockの作成を始める（チェックはないのでいつでもblockを作成できるが、ブロックチェーンではないのか）

- Nodeが複数いて，異なるBlockChainを持つときは，どのように正しさを判断するか？
    - **Consensus Algorithm**
    - 最も長いblockchainを，最も正しいblockchainとみなす
        - なぜなら，chainが長ければ長いほど，多くの計算資源が注ぎ込まれたことになるから
        - 多数決ではなく，**one-CPU-one-vote**

- Nodeの役割を考え直す必要がある

- Nodeが複数いて、異なる分散台帳を持つが、それらは論理的に単一である
    - **Consensus Algorithm** は簡単なものであり、許可型の分散台帳において、論理的な単一性を保持する

In [7]:
# from uuid import uuid4
# from time import sleep
# from copy import deepcopy
# import multiprocessing as mp
# import random
# from itertools import count

- nodeを実装する
- networkまわりの細かい作業は，network interfaceに任せる

- 改めて、いわゆるマイニングを必要とするノードにおいては、ノードが複雑であると思われる。
- コンフリクトは起こらない（２つのコンセンサスアルゴリズムを用意して、その様子を比較検討できるようにする）

In [8]:
from node import Node

- 全体的に，networkの作業が追加されている
- mineを少し変更している
    - 複数のnodeが同じ方法でマイニングをするのは無駄が生じる可能性がある
        - 例えば全く同じBlockのマイニングを複数のノードが開始した場合，確実に最も計算力の高いnodeがマイニングを成功させることになる

- Walletもネットワーク対応に
    - transactionを発行したら，知っているnodeにtransactionを送る

- networkっぽいものを実装する
    - Network classを中継点としてデータの遣り取りをする
- あとでmultiprocessでノードを動かすために，データの管理はmultiprocess.managerで行う
- flaskとかで作っても面白いかもしれない
- 本質的でないので読み飛ばして良い

## ノード1つの場合のテスト

In [9]:
genesis=Block(time(), (), "0")
uuid='the_leader_node'
network=Network({uuid:[uuid]})
node=Node(network, genesis, uuid)

### トランザクションはライブラリ化される必要がある

### 今回可読性を重要視したため、暗号化は試用していない

In [10]:
alice=Wallet(network, [uuid])
bob=Wallet(network, [uuid])

- これを実行してしばらく問題が起きなければOK
    - 適当にinterruptする

In [11]:
node.work(verbose=False)

KeyboardInterrupt: 


- 後片付け

In [12]:
network.shutdown()

# デモ: 分散ブロックチェーンのデモ
- multiprocessingでネットワークのデモを行う

In [13]:
genesis=Block(time(), (), "0")
network=Network()

- ノードを生成する
    - ノードにはデモで分かりやすいように名前を与えてある

In [14]:
nodes=['the_leader_node', 'the_blue_sky', 'white_noise', 'yellow_submarine', 'green_planet']


- nodeのnetworkの状態を定義する
- 序列を持つ結合と全結合を選択できる
    - リーダーノードは他のノードと結合しているが、その他のノードはリーダーとのみ結合している
    - 全結合の場合は全てのノード同士で直接やりとりできる

（考察）
- 1つのnodeはただ１つの決まった結合を持つ場合、コンフリクトは起こらない
- ブロックが作成されるたびにリーダーにブロードキャストしたい

In [15]:
ORDERING=True
if (ORDERING):
    network.neighbours['the_leader_node']=['the_leader_node', 'the_blue_sky', 'white_noise', 'yellow_submarine', 'green_planet']
    network.neighbours['the_blue_sky']=['the_leader_node', 'the_blue_sky']
    network.neighbours['white_noise']=['the_leader_node', 'white_noise']
    network.neighbours['yellow_submarine']=['the_leader_node', 'yellow_submarine']
    network.neighbours['green_planet']=['the_leader_node', 'green_planet']
else:
    for uuid in nodes:
        network.neighbours[uuid]=nodes

- multiprocess workerを作って起動する

In [16]:
def node_work(uuid):
    node=Node(network, genesis, uuid)
    node.work()

In [17]:
workers=[]
for uuid in nodes:
    worker=mp.Process(target=node_work, args=(uuid,))
    workers.append(worker)
    worker.start()

- peopleを生成する
- これも適当に人数を変えて遊ぶと良い

- peopleは概念としては今回はあまり試用していない

In [18]:
# num_people=2
# people=[Wallet(network, random.sample(nodes, random.randint(1, len(nodes)))) for _ in range(num_people)]
alice=Wallet(network, ['the_leader_node'])
bob=Wallet(network, ['the_blue_sky'])

people=[alice, bob]

### ブロックチェーンの表示
- 各ノードの持つブロックチェーンがどの様になっているのか表示する

In [ ]:
from IPython.display import clear_output
import time

timeout = time.time() + 60
def draw_chains(network):
    while True:
        if time.time() > timeout:
            break
        line=""
        for k, v in network.chains.items():
            line += f"(Node: {k[:15]}) "
            chain=BlockChain.json_loads(v)
            h=list(map(lambda x: x.hash()[:5], chain))
            line += "--".join(map(lambda x: f"[{x}]", h))
            line += "\n"
        print(line.rstrip())
        clear_output(True)
        sleep(1.0)

worker=mp.Process(target=draw_chains, args=(network,))
worker.start()

(Node: the_leader_node) [a452b]--[24289]--[6a1cb]--[927cc]--[dee79]
(Node: the_blue_sky) [a452b]--[24289]--[6a1cb]--[927cc]--[dee79]
(Node: white_noise) [a452b]--[24289]--[6a1cb]--[927cc]--[dee79]
(Node: yellow_submarin) [a452b]--[24289]--[6a1cb]--[927cc]--[dee79]
(Node: green_planet) [a452b]--[24289]--[6a1cb]--[927cc]--[dee79]


- 取引をして遊ぶ
    - ランダムに取引が行われるコードを書いてみる
    - 上の表示が変化する!

In [20]:
def generate_transactions():
    # for _ in range(random.randint(1,10)):
    #     sender, receiver= random.sample(people, 2)
    #     # sender.send(receiver.address, random.randint(1, 100))
    #     sender.send(receiver.address, 'inquiry')

    alice.send([None], 'cancel')
    bob.send([None], 'reservation')

(Node: the_leader_node) [a452b]
(Node: the_blue_sky) [a452b]
(Node: white_noise) [a452b]
(Node: yellow_submarin) [a452b]
(Node: green_planet) [a452b]


In [21]:
# generate_transactions()

### スマートコントラクトの実装に向けて

### メッセージを出力した方が良いかもしれない

In [22]:
sc = Smartcontract(network)

s_uuid = 'the_leader_node'
s = Wallet(network, [s_uuid])
b_uuid = 'the_blue_sky'
b = Wallet(network, [b_uuid])

the_leader_node は離着陸地点を予約しました

In [23]:
clear_output(True)

print(s_uuid + ' は離着陸地点を予約しました。' if sc.make_a_reservation(s) else s_uuid + ' は離着陸地点は予約できませんでした。')

the_leader_node は離着陸地点を予約しました。


In [24]:
clear_output(True)

print('\n以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

print('\n以下ブロックチェーンの源蔵のステート')
print(sc.get_current_state(s_uuid))


以下ブロックチェーンの内容（トランザクションの履歴）
0 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "reservation", "sign": null}

以下ブロックチェーンの源蔵のステート
{'sender_address': ['the_leader_node'], 'receiver_address': [None], 'reserved': 1}


In [25]:
clear_output(True)

print(s_uuid + ' により離着陸地点の予約はキャンセルされました。' if sc.cancel_reservation(s) else s_uuid + ' によっては離着陸地点はキャンセルできませんでした。')

the_leader_node により離着陸地点の予約はキャンセルされました。


In [26]:
clear_output(True)

print('\n以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

print('\n以下ブロックチェーンの源蔵のステート')
print(sc.get_current_state(s_uuid))


以下ブロックチェーンの内容（トランザクションの履歴）
0 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "reservation", "sign": null}
1 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "cancel", "sign": null}

以下ブロックチェーンの源蔵のステート
{'sender_address': None, 'receiver_address': None, 'reserved': 0}


In [27]:
clear_output(True)

print(b_uuid + ' は離着陸地点を予約しました。' if sc.make_a_reservation(b) else b_uuid + ' は離着陸地点は予約できませんでした。')

the_blue_sky は離着陸地点を予約しました。


In [28]:
clear_output(True)

print('\n以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

print('\n以下ブロックチェーンの源蔵のステート')
print(sc.get_current_state(s_uuid))


以下ブロックチェーンの内容（トランザクションの履歴）
0 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "reservation", "sign": null}
1 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "cancel", "sign": null}
2 {"sender_address": ["the_blue_sky"], "receiver_address": [null], "value": "reservation", "sign": null}

以下ブロックチェーンの源蔵のステート
{'sender_address': ['the_blue_sky'], 'receiver_address': [None], 'reserved': 1}


In [29]:
clear_output(True)

print(b_uuid + ' により離着陸地点の予約はキャンセルされました。' if sc.cancel_reservation(b) else b_uuid + ' によっては離着陸地点はキャンセルできませんでした。')

the_blue_sky により離着陸地点の予約はキャンセルされました。


In [31]:
clear_output(True)

print('\n以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

print('\n以下ブロックチェーンの源蔵のステート')
print(sc.get_current_state(s_uuid))


以下ブロックチェーンの内容（トランザクションの履歴）
0 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "reservation", "sign": null}
1 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "cancel", "sign": null}
2 {"sender_address": ["the_blue_sky"], "receiver_address": [null], "value": "reservation", "sign": null}
3 {"sender_address": ["the_blue_sky"], "receiver_address": [null], "value": "cancel", "sign": null}

以下ブロックチェーンの源蔵のステート
{'sender_address': None, 'receiver_address': None, 'reserved': 0}


In [32]:
clear_output(True)

print(b_uuid + ' により離着陸地点の予約はキャンセルされました。' if sc.cancel_reservation(b) else b_uuid + ' によっては離着陸地点はキャンセルできませんでした。')

print('\n以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

print('\n以下ブロックチェーンの源蔵のステート')
print(sc.get_current_state(s_uuid))

the_blue_sky によっては離着陸地点はキャンセルできませんでした。

以下ブロックチェーンの内容（トランザクションの履歴）
0 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "reservation", "sign": null}
1 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "cancel", "sign": null}
2 {"sender_address": ["the_blue_sky"], "receiver_address": [null], "value": "reservation", "sign": null}
3 {"sender_address": ["the_blue_sky"], "receiver_address": [null], "value": "cancel", "sign": null}

以下ブロックチェーンの源蔵のステート
{'sender_address': None, 'receiver_address': None, 'reserved': 0}


### 片付け

In [ ]:
worker.terminate()
worker.kill()
for worker in workers:
    worker.terminate()
    worker.kill()
network.shutdown()

# おわりに
- 本物のBitCoinとは異なる部分がたくさんあります

# 参考文献
- [Python で Blockchain の実装をする](https://github.com/tamuhey/python_blockchain)
    - たいへん参考にさせていただきました
- [Bitcoin white paper](https://bitcoin.org/bitcoin.pdf)
    - 専門外でも読みやすいです